In [15]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

import altair as alt
import geopandas as gpd
import pandas as pd
import toolz

import ipywidgets as widgets
from ipywidgets import interactive, VBox, HBox
from IPython.display import display, clear_output

def custom(data):
    return toolz.curried.pipe(data, alt.to_json(filename='altdata/{prefix}-{hash}.{extension}') )
alt.data_transformers.register('custom', custom)
alt.data_transformers.enable('custom')


DataTransformerRegistry.enable('custom')

In [3]:
# Load the shapefile
shp_data = gpd.read_file('departements-version-simplifiee.geojson')
shp_data = shp_data[shp_data['nom'] != 'NaN']

shp_data['centroid_x'] = shp_data.geometry.centroid.x
shp_data['centroid_y'] = shp_data.geometry.centroid.y

# Create an Altair chart from the geoshape object
chart = alt.Chart(shp_data).mark_geoshape(
  fill='grey', stroke='white', strokeWidth=0.5
).properties(
  width = 700,
  height = 700
)

# Display the chart
chart

C:\Users\jlare\AppData\Local\Temp\ipykernel_9884\218244799.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shp_data['centroid_x'] = shp_data.geometry.centroid.x
C:\Users\jlare\AppData\Local\Temp\ipykernel_9884\218244799.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shp_data['centroid_y'] = shp_data.geometry.centroid.y


alt.Chart(...)

In [4]:
names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)
names.drop(names[names.dpt == 'Nan'].index, inplace=True)

names.sample(5)

,sexe,preusuel,annais,dpt,nombre
2378701,2,EMMANUELLE,1971,22,23
56608,1,ALANN,1997,29,3
1450073,1,ROGER,1953,78,63
2493271,2,GABRIELLE,2001,45,10
2057953,2,CARLA,2000,64,15


In [5]:
# Select most given names

ids = names.groupby(['dpt','annais','sexe'])['nombre'].idxmax()
most_given_names = names.loc[ids]
most_given_names.sample(10)

,sexe,preusuel,annais,dpt,nombre
1319908,1,PATRICK,1962,84,147
1341423,1,PHILIPPE,1966,42,400
3052126,2,MARIE,1930,24,214
1461142,1,ROMAIN,1991,12,41
2782924,2,LAURA,1992,73,55
3053430,2,MARIE,1944,12,138
2377187,2,EMMA,2017,54,58
775619,1,JEAN,1952,47,191
3620414,2,SYLVIE,1963,81,145
772106,1,JEAN,1915,12,90


In [6]:
# Merge with geoshape
global_gdf = shp_data.merge(most_given_names, how='right', left_on='code', right_on='dpt')



global_gdf.sample(50)

,code,nom,geometry,centroid_x,centroid_y,sexe,preusuel,annais,dpt,nombre
18656,78,Yvelines,"POLYGON ((2.20059 48.90868, 2.16838 48.89508, ...",1.841634,48.815414,1,ANDRÉ,1911,78,347
22984,NaN,NaN,None,NaN,NaN,1,KEVIN,1995,973,41
17628,73,Savoie,"POLYGON ((6.80252 45.77837, 6.80842 45.72515, ...",6.442871,45.477880,1,LUCAS,2002,73,66
19436,81,Tarn,"POLYGON ((1.99017 44.14945, 2.02970 44.15704, ...",2.165614,43.785591,1,JEAN,1938,81,213
21839,91,Essonne,"POLYGON ((2.22656 48.77610, 2.23298 48.76620, ...",2.242750,48.521970,2,LÉA,1997,91,171
15780,66,Pyrénées-Orientales,"POLYGON ((2.16605 42.66392, 2.17620 42.65251, ...",2.521176,42.600156,1,JEAN,1925,66,179
15550,65,Hautes-Pyrénées,"MULTIPOLYGON (((-0.10308 43.24282, -0.12194 43...",0.163799,43.053761,1,JEAN,1931,65,245
5138,22,Côtes-d'Armor,"POLYGON ((-3.65914 48.65921, -3.63649 48.67069...",-2.865489,48.440506,1,JEAN,1928,22,571
18911,79,Deux-Sèvres,"POLYGON ((-0.89196 46.97582, -0.85592 46.97908...",-0.318088,46.557049,2,MARIE,1917,79,130
9709,41,Loir-et-Cher,"POLYGON ((0.84122 48.10306, 0.87589 48.10944, ...",1.428916,47.616948,2,MADELEINE,1914,41,122


In [7]:
year = 1989
subset = global_gdf[global_gdf['annais'] == str(year)]

# alt.Chart(global_gdf).mark_geoshape(stroke='white').encode(
#     color=alt.Max('nombre'),
# ).properties(width=800, height=600)

# Separate subsets for boys and girls
subset_girls = subset[subset['sexe'] == 2]
subset_boys = subset[subset['sexe'] == 1]
subset_girls.sample(10)


# Merge boys and girls data to create tooltip information
tooltip_data = subset_girls[['code', 'preusuel']].merge(
    subset_boys[['code', 'preusuel']],
    on='code',
    suffixes=('_girl', '_boy')
)

# Merge tooltip data back with shape data
merged_data = shp_data.merge(tooltip_data, how='left', on='code')

merged_data.sample(10)

# Create Altair chart
chart = alt.Chart(merged_data).mark_geoshape(stroke='white').encode(
    tooltip=[alt.Tooltip('nom:N', title='Region'),
             alt.Tooltip('preusuel_girl:N', title='Most Given Name (Girl)'),
             alt.Tooltip('preusuel_boy:N', title='Most Given Name (Boy)')],
    color=alt.value('lightgrey')
).properties(
    width=800,
    height=600
)

seuil_minimal = 0.25

# Filtrer les départements par taille
filtered_data = merged_data[merged_data['geometry'].area > seuil_minimal]

# Labels pour les prénoms des filles (filtre appliqué)
text_girl = alt.Chart(filtered_data).mark_text(
    align='center',
    baseline='middle',
    fontSize=10,
    color='red'
).encode(
    longitude='centroid_x:Q',
    latitude='centroid_y:Q',
    text='preusuel_girl:N'
)

# Labels pour les prénoms des garçons (filtre appliqué)
text_boy = alt.Chart(filtered_data).mark_text(
    align='center',
    baseline='middle',
    dy=10,  # Décalage vertical
    fontSize=10,
    color='blue'
).encode(
    longitude='centroid_x:Q',
    latitude='centroid_y:Q',
    text='preusuel_boy:N'
)

# Display the chart
chart + text_girl + text_boy

C:\Users\jlare\AppData\Local\Temp\ipykernel_9884\3438070681.py:40: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  filtered_data = merged_data[merged_data['geometry'].area > seuil_minimal]


alt.LayerChart(...)

In [8]:
def createMaps(year):

    subset = global_gdf[global_gdf['annais'] == str(year)]

    # Separate subsets for boys and girls
    subset_girls = subset[subset['sexe'] == 2]
    subset_boys = subset[subset['sexe'] == 1]

    # Merge boys and girls data to create tooltip information
    tooltip_data = subset_girls[['code', 'preusuel']].merge(
        subset_boys[['code', 'preusuel']],
        on='code',
        suffixes=('_girl', '_boy')
    )

    # Merge tooltip data back with shape data
    merged_data = shp_data.merge(tooltip_data, how='left', on='code')

    chartG = alt.Chart(merged_data).mark_geoshape(stroke='white').encode(
        tooltip=[alt.Tooltip('nom:N', title='Region'),
                alt.Tooltip('preusuel_girl:N', title='Most Given Name (Girl)')],
        color=alt.Color('preusuel_girl:N')
    ).properties(
        width=800,
        height=600
    )

    chartB = alt.Chart(merged_data).mark_geoshape(stroke='white').encode(
        tooltip=[alt.Tooltip('nom:N', title='Region'),
                alt.Tooltip('preusuel_boy:N', title='Most Given Name (Boy)')],
        color=alt.Color('preusuel_boy:N')
    ).properties(
        width=800,
        height=600
    )

    seuil_minimal = 0.25

    # Filtrer les départements par taille
    filtered_data = merged_data[merged_data['geometry'].area > seuil_minimal]

    # Labels pour les prénoms des filles (filtre appliqué)
    text_girl = alt.Chart(filtered_data).mark_text(
        align='center',
        baseline='middle',
        fontSize=10,
        color='black'
    ).encode(
        longitude='centroid_x:Q',
        latitude='centroid_y:Q',
        text='preusuel_girl:N'
    )

    text_boy = alt.Chart(filtered_data).mark_text(
        align='center',
        baseline='middle',
        dy=10,  # Décalage vertical
        fontSize=10,
        color='black'
    ).encode(
        longitude='centroid_x:Q',
        latitude='centroid_y:Q',
        text='preusuel_boy:N'
    )

    # Display the chart
    return (chartG + text_girl) & (chartB + text_boy)

In [16]:
year_slider = widgets.IntSlider(value=2010, min=1900, max=2020, step=1, description='Year')

# Conteneur pour le graphique
output = widgets.Output()


def update_chart(year):
    with output:
        clear_output(wait=True)
        chart = createMaps(year)
        display(chart)

# Configurer l'interaction
interactive_plot = interactive(update_chart, year=year_slider)

# Layout pour éviter la duplication des widgets
layout = VBox([HBox([year_slider]), output])

# Affichage
display(layout)

# Initialiser le graphique avec les valeurs par défaut
update_chart(year_slider.value)